In [1]:
import cv2
import numpy as np
import math
import os
from PIL import Image, ImageFont, ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(suppress=True)
import time
import subprocess

In [2]:
import yolo_3_anchors as yolo
chleba = yolo.YOLO(model_path='D:/data-shared/tpami_paper/cityscapes/nr_of_anchors/detector/3_anchors_143814.h5')
file = open('inference_nr_of_anchors_3.txt', "w")

Using TensorFlow backend.
W0506 09:32:04.189337   416 module_wrapper.py:139] From C:\Users\Hurtik\python-petr\yolov4\tpami_paper\cityscapes\nr_of_anchors\yolo_3_anchors.py:40: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0506 09:32:04.190336   416 module_wrapper.py:139] From C:\Users\Hurtik\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0506 09:32:04.191312   416 module_wrapper.py:139] From C:\Users\Hurtik\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0506 09:32:04.191312   416 module_wrapper.py:139] From C:\Users\Hurtik\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:203: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0506 09:32:05.016338   416 modul

D:/data-shared/tpami_paper/cityscapes/nr_of_anchors/detector/3_anchors_143814.h5 model, anchors, and classes loaded.


In [3]:
with open('cityscapes-val-small.txt') as f:
    lines = f.readlines()


total_boxes = 0
imgs = 0
start = time.time()

for im in range (1, len(lines), 1):
    imgs += 1
    aaa = lines[im].split()
    img   = Image.open(aaa[0])
    
    boxes   = []
    scores  = []
    classes = []
    box, score, classs, polygons = chleba.detect_image(img)
    
    
    for k in range (0, len(box)):
        boxes.append((box[k][1], box[k][0], box[k][3], box[k][2]))
        scores.append(score[k])
        classes.append(classs[k])
        
    
    total_boxes += len(boxes)
    if len(boxes)==0:
        continue
    
    file.write(aaa[0]+" ") 
    for b in range(0, len(boxes)):
        str_to_write = ''
        
        str_to_write += str(float(boxes[b][0]))+","+str(float(boxes[b][1]))+","+str(float(boxes[b][2]))+","+str(float(boxes[b][3]))+","
        str_to_write += str(scores[b])+","
        str_to_write += str(int(classes[b]))
        
        offset      = len(polygons[b])//3 # 72 for 24 vertexes. offset = 24
        vertices    = 0
        for dst in range(0, len(polygons[b])//3): # 下取整
            if polygons[b][dst+offset*2] > 0.2:
                str_to_write += ","+str(float(polygons[b][dst]))+","+str(float(polygons[b][dst+offset]))
                vertices+=1
        str_to_write +=  " "
        if vertices < 3:
            print('found not correct polygon with ', vertices, ' vertices')
            continue
        file.write(str_to_write)
        
    file.write("\n")    
    
file.close()    
print('total boxes: ', total_boxes)
print('imgs: ', imgs)
end = time.time()
print('fps: ', imgs/(end - start))

total boxes:  1027
imgs:  99
fps:  7.146077495571263
